## Conversation History Memory Integration with Calendars

**Derek Rosenzweig** 

This notebook converts structured memory traces (stored as `.json`) into `.ics` calendar files, enabling users to visualize and revisit key events — particularly conversation history with language models — within their preferred calendar tools (Apple Calendar, Google Calendar, Outlook). The goal is to make memory traces both queryable and actionable by aligning them with familiar, time-based interfaces that users trust and regularly engage with.

##### Use Cases for Calendar-Based Memory 

By ingesting .ics files from tools like Google Calendar or Apple Calendar, agents can:

- Ground responses in the user’s actual schedule and historical events

- Retrieve relevant context by aligning timestamps and event metadata

- Prioritize upcoming tasks or recall previous interactions based on calendar entries

This workflow supports persistent, time-aware context grounding—enabling language models to condition their outputs based on the user's routines, workflows, and conversation history.

##### What This Example Covers 

- Represent memory traces (ie. conversation history) in structured `.json`
- Converting traces into `.ics` format for calendar input 
- Visualizing `.ics` in familiar calendar interfaces 
- Use cases for planning, agent orchestration, and scheduling

In [10]:
from pathlib import Path
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv(dotenv_path=Path(".env"))

GOOGLE_CREDENTIALS_PATH = os.getenv("GOOGLE_CREDENTIALS_PATH")  # e.g., "calendar/credentials.json"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # For later use, if needed

assert GOOGLE_CREDENTIALS_PATH is not None, "Missing Google credentials path"


AssertionError: Missing Google credentials path


### 1. Define Memory Traces from Conversation History 

Each trace is a timestamped unit of memory (goal, reflection, event) with fields for content, title, location, importance, and more.


In [4]:
# Example conversation history memory traces 

memory_traces = {
    "user_id": "derek_dev",
    "session_id": "cookbook_walkthrough",
    "memory": [
        {
            "id": "m101",
            "type": "goal",
            "timestamp": "2025-04-10T08:30:00Z",
            "title": "Design JSON Schema for Memory Traces",
            "content": "Finalize the structured schema to capture conversation goals, reflections, and observations.",
            "location": "SF Office"
        },
        {
            "id": "m102",
            "type": "reflection",
            "timestamp": "2025-04-12T15:00:00Z",
            "title": "Initial Demo Success",
            "content": "Successfully generated ICS from memory traces and confirmed Apple Calendar compatibility.",
            "location": "Remote Workspace"
        },
        {
            "id": "m103",
            "type": "goal",
            "timestamp": "2025-04-13T10:15:00Z",
            "title": "Send Cookbook to Collaborators",
            "content": "Package and share demo with OpenAI and Google collaborators for feedback."
        }
    ]
}

print(memory_traces)

{'user_id': 'derek_dev', 'session_id': 'cookbook_walkthrough', 'memory': [{'id': 'm101', 'type': 'goal', 'timestamp': '2025-04-10T08:30:00Z', 'title': 'Design JSON Schema for Memory Traces', 'content': 'Finalize the structured schema to capture conversation goals, reflections, and observations.', 'location': 'SF Office'}, {'id': 'm102', 'type': 'reflection', 'timestamp': '2025-04-12T15:00:00Z', 'title': 'Initial Demo Success', 'content': 'Successfully generated ICS from memory traces and confirmed Apple Calendar compatibility.', 'location': 'Remote Workspace'}, {'id': 'm103', 'type': 'goal', 'timestamp': '2025-04-13T10:15:00Z', 'title': 'Send Cookbook to Collaborators', 'content': 'Package and share demo with OpenAI and Google collaborators for feedback.'}]}


### 2. Convert JSON to `.ics`

This function converts each trace to a valid calendar event.

See ICS formatting standards

In [7]:
from datetime import datetime, timedelta

# Function to convert ISO datetime to iCalendar format
def datetime_to_ics(dt: str) -> str:
    """Convert ISO 8601 datetime string to iCalendar format."""
    return datetime.fromisoformat(dt.replace("Z", "+00:00")).strftime("%Y%m%dT%H%M%SZ")

# Function to generate iCalendar event string from a memory trace
def generate_ics_string(trace: dict) -> str:
    """
    Convert a memory trace to iCalendar event format.
    
    Parameters:
    - trace: A dictionary containing memory trace data with fields:
        - id: Unique identifier for the memory
        - timestamp: ISO 8601 datetime string
        - title: Event title
        - content: Detailed description
        - location: Optional location string
    
    Returns:
    - String in iCalendar VEVENT format
    """
    start_dt = datetime.fromisoformat(trace["timestamp"].replace("Z", "+00:00"))
    end_dt = start_dt + timedelta(minutes=15)  # Default 15-minute duration

    summary = trace["title"]
    description = trace["content"]
    location = trace.get("location", "")
    uid = f"{trace['id']}@memorysystem.ai"

    return f"""BEGIN:VEVENT
UID:{uid}
DTSTAMP:{datetime_to_ics(trace['timestamp'])}
DTSTART:{datetime_to_ics(trace['timestamp'])}
DTEND:{datetime_to_ics(end_dt.isoformat())}
SUMMARY:{summary}
DESCRIPTION:{description}
LOCATION:{location}
STATUS:CONFIRMED
END:VEVENT
"""

# Process all memory traces and convert to iCalendar events
print("Converting memory traces to iCalendar format...")
ics_events = [generate_ics_string(trace) for trace in memory_traces["memory"]]

# Create the complete iCalendar content as a string variable for later use
ics_content = "BEGIN:VCALENDAR\nVERSION:2.0\nPRODID:-//MemorySystem//EN\n"
ics_content += "".join(ics_events)
ics_content += "END:VCALENDAR"

# Write the content to a file
with open("calendar_memory_demo.ics", "w") as f:
    f.write(ics_content)

# Test and print the output of the functions
print("Testing datetime_to_ics function:")
for trace in memory_traces["memory"]:
    print(f"Original timestamp: {trace['timestamp']}")
    print(f"Converted to iCalendar format: {datetime_to_ics(trace['timestamp'])}\n")

print("Testing generate_ics_string function:")
for trace in memory_traces["memory"]:
    ics_string = generate_ics_string(trace)
    print(f"Memory Trace ID: {trace['id']}")
    print("Generated iCalendar Event:")
    print(ics_string)
    print("\n" + "-"*40 + "\n")

print("iCalendar file 'calendar_memory_demo.ics' has been created.")
print("The iCalendar content is also stored in the 'ics_content' variable for later use.")

Converting memory traces to iCalendar format...
Testing datetime_to_ics function:
Original timestamp: 2025-04-10T08:30:00Z
Converted to iCalendar format: 20250410T083000Z

Original timestamp: 2025-04-12T15:00:00Z
Converted to iCalendar format: 20250412T150000Z

Original timestamp: 2025-04-13T10:15:00Z
Converted to iCalendar format: 20250413T101500Z

Testing generate_ics_string function:
Memory Trace ID: m101
Generated iCalendar Event:
BEGIN:VEVENT
UID:m101@memorysystem.ai
DTSTAMP:20250410T083000Z
DTSTART:20250410T083000Z
DTEND:20250410T084500Z
SUMMARY:Design JSON Schema for Memory Traces
DESCRIPTION:Finalize the structured schema to capture conversation goals, reflections, and observations.
LOCATION:SF Office
STATUS:CONFIRMED
END:VEVENT


----------------------------------------

Memory Trace ID: m102
Generated iCalendar Event:
BEGIN:VEVENT
UID:m102@memorysystem.ai
DTSTAMP:20250412T150000Z
DTSTART:20250412T150000Z
DTEND:20250412T151500Z
SUMMARY:Initial Demo Success
DESCRIPTION:Successf

In [9]:
ics_content 

'BEGIN:VCALENDAR\nVERSION:2.0\nPRODID:-//MemorySystem//EN\nBEGIN:VEVENT\nUID:m101@memorysystem.ai\nDTSTAMP:20250410T083000Z\nDTSTART:20250410T083000Z\nDTEND:20250410T084500Z\nSUMMARY:Design JSON Schema for Memory Traces\nDESCRIPTION:Finalize the structured schema to capture conversation goals, reflections, and observations.\nLOCATION:SF Office\nSTATUS:CONFIRMED\nEND:VEVENT\nBEGIN:VEVENT\nUID:m102@memorysystem.ai\nDTSTAMP:20250412T150000Z\nDTSTART:20250412T150000Z\nDTEND:20250412T151500Z\nSUMMARY:Initial Demo Success\nDESCRIPTION:Successfully generated ICS from memory traces and confirmed Apple Calendar compatibility.\nLOCATION:Remote Workspace\nSTATUS:CONFIRMED\nEND:VEVENT\nBEGIN:VEVENT\nUID:m103@memorysystem.ai\nDTSTAMP:20250413T101500Z\nDTSTART:20250413T101500Z\nDTEND:20250413T103000Z\nSUMMARY:Send Cookbook to Collaborators\nDESCRIPTION:Package and share demo with OpenAI and Google collaborators for feedback.\nLOCATION:\nSTATUS:CONFIRMED\nEND:VEVENT\nEND:VCALENDAR'